## INTEGRACIÓN PARA CONSTRUCCIÓN DE UN GOLDEN DATASET CON API GPT-3.5 Turbo

In [3]:
from openai import OpenAI
import os
import pathlib 
import pandas as pd
import ast
from dotenv import load_dotenv
from collections import defaultdict

#pip install --upgrade openai==1.1.1
#pip install cohere tiktoken
#!pip3 install python-dotenv

#print(openai.__version__)

In [6]:
ruta_archivo = '/export/usuarios_ml4ds/cggamella/RAG_tool/dataset_pruebas.xlsx'
# Utiliza la función to_excel para exportar el DataFrame a un archivo Excel
df = pd.read_excel(ruta_archivo)

In [7]:
df

,acronyms,expanded_acronyms,text,detected_acronyms,LLM_expanded_acronyms
0,ONG,ONG,Las ONG son organizaciones no gubernamentales ...,NaN,NaN
1,"CIA, FBI","CIA, FBI",La CIA y el FBI colaboran en investigaciones i...,NaN,NaN
2,"UE, OTAN","UE, OTAN",La unión europea y la OTAN tienen roles crucia...,NaN,NaN
3,NASA,NASA,"La NASA es la agencia espacial estadounidense,...",NaN,NaN
4,UNICEF,UNICEF,"UNICEF, trabajan por los derechos de los niños...",NaN,NaN
...,...,...,...,...,...
63,FAQ,FAQ,La sección de preguntas frecuentes FAQ proporc...,NaN,NaN
64,CEO,CEO,El CEO de la compañía anunció planes de expans...,NaN,NaN
65,CFO,CFO,El chief financial officer presentó el informe...,NaN,NaN
66,CTO,CTO,El CTO lidera el equipo de desarrollo tecnológ...,NaN,NaN


Integración OpenAI

Esta función sirve para generar las queries asociadas a los servicios, cat y subcat con el modelo de GPT-3.5 Turbo, además se crea una función auxiliar para crear la base de datos escribiendo en el campo necesario la información de la Categoría, Subcategoría, Servicio, Query. 

In [ ]:
def generar_queries_servicios(category_dict):
    
    data = [] 

    for categoria, subcategorias in category_dict.items():
        for subcategoria, servicios in subcategorias.items():
            for servicio in servicios:
                try:
                    completion = openai.chat.completions.create(
                        model="gpt-3.5-turbo-0125",
                        messages=[
                          {"role": "system", "content": "You are a nice and helpful assistant.I need your imagination to create some queries that a human can send to find a service or professional. Language used must be Spanish."},
                          {"role": "user", "content": f"Generate 5 examples of queries that a user without specific knowledge could make describing his issue looking for services related to the service name: {servicio},which belongs to subcategory: {subcategoria}, which is part of the category {categoria}. Each query should be no longer than 8 words, not being enumerated neither with quotation marks, provide plain text"}
                        ],
                        temperature = 0.7, #Es configurable entre 0 y 2, cuanto más alto sea, el modelo es más creativo
                        max_tokens = 300 # Para limitar el número de tokens, aproximadamente 100 tokens son 75 palabras
                    )
                    # Acceder directamente al texto generado y dividirlo en líneas para obtener las queries
                    queries = completion.choices[0].message.content.strip().split('\n')
                    
                    return queries
                
                except Exception as e:
                    print(f"Se produjo un error al generar las queries para el servicio {servicio} en {subcategoria} de {categoria}: {e}")
                    # Continúa con el siguiente servicio en caso de error
                    continue

def generar_tabla_queries(category_dict):

    data = []

    # Itera sobre el diccionario de categorías
    for categoria, subcategorias in category_dict.items():
        for subcategoria, servicios in subcategorias.items():
            for servicio in servicios:
                # Obtiene las queries para el servicio actual
                queries = generar_queries_servicios({categoria: {subcategoria: [servicio]}})
                # Añade cada query al listado de datos
                for query in queries:
                    data.append({
                        "Categoría": categoria,
                        "Subcategoría": subcategoria,
                        "Servicio": servicio,
                        "Query": query
                    })

    # Crea un DataFrame con los datos acumulados
    df_queries = pd.DataFrame(data)

    # Devuelve el DataFrame
    return df_queries


# Genera el DataFrame con las queries
df_queries = generar_tabla_queries(category_dict)

# Guarda el DataFrame en un archivo Excel
df_queries.to_excel("queries_sinteticas_servicios.xlsx", index=False)

Ha tardado 1h 8 mins en generar 5205 queries

Leyendo el archivo de queries_sintéticas creado

In [ ]:
df_bbdd = pd.read_excel("queries_sinteticas_servicios.xlsx")

# Integración para acronyms

In [ ]:
#!pip install git+https://github.com/stanfordnlp/dspy.git

In [ ]:
!pip list

In [ ]:
#!pip install --upgrade pydantic
#!pip install --upgrade openai

In [14]:
import pathlib
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import random
import os
import re
import dspy
from dotenv import load_dotenv
import pickle

In [15]:
# Clase para dar formato al output del terminal
class Colors:
    BLUE = '\033[94m'
    ENDC = '\033[0m'

In [16]:
###########
#   LLM   #
###########
lm = dspy.HFClientTGI(model="meta-llama/Meta-Llama-3-8B ", port=8080, url="http://127.0.0.1")
dspy.settings.configure(lm=lm)

In [68]:
qa = dspy.ChainOfThought('question -> answer')

# Run with Llama instead
with dspy.context(lm=lm):
    response = qa(question="What is the first day of the week?")
    print('Meta-Llama-3-8B:', response.answer)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: /generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa384863b80>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [9]:
# Cargar dataframe donde se guardará la información 
ruta_archivo = '/export/usuarios_ml4ds/cggamella/RAG_tool/dataset_pruebas.xlsx'
df = pd.read_excel(ruta_archivo)

In [10]:
lista_acronyms = set(df['acronyms'].str.lower().tolist())

In [69]:
class AcronymGenerator(dspy.Signature):
    """
    Generar un texto extenso para los acrónimos dados, asegurándose de que contenga los acrónimos dentro de un contexto simulado.
    
    Ejemplo:
    --------
    ACRÓNIMOS: ["ONU", "NASA"]
    TEXTO_GENERADO: "En la última reunión de la ONU, se discutieron los avances tecnológicos presentados por la NASA.
    La ONU destacó la importancia de estos desarrollos para la exploración espacial y la cooperación internacional..."
    """
    
    ACRONIMOS = dspy.InputField(desc="List of acronyms in spanish that must be used to generate realistic text")
    TEXTO_GENERADO = dspy.OutputField(desc="List of acronyms in spanish that must be used to generate realistic text")
    
class AcronymGeneratorModule(dspy.Module):
    def __init__(self, model_url, model_port):
        super().__init__()
        self.generator = dspy.ChainOfThought(AcronymGenerator)
        self.lm = dspy.HFClientTGI(model="meta-llama/Meta-Llama-3-8B", port=model_port, url=model_url)
        dspy.settings.configure(lm=self.lm)

    def forward(self, acronyms):
        prompt = self.create_prompt(acronyms)
        qa = dspy.ChainOfThought('question -> answer')

        # Utilizando el contexto de dspy con LLaMA
        with dspy.context(lm=self.lm):
            response = qa(question=prompt)
        
        generated_text = response.answer.strip()
        return dspy.Prediction(TEXTO_GENERADO=generated_text)

    def create_prompt(self, acronyms):
        acronyms_str = ", ".join(acronyms)
        prompt = (
            f"Genera un texto en español que incluya los siguientes acrónimos en un contexto realista y detallado: {acronyms_str}. "
            "El texto debe ser extenso y desarrollar un escenario en el cual estos acrónimos sean relevantes. "
            "Por favor, asegúrate de que el texto sea coherente y bien estructurado."
        )
        return prompt

In [60]:
acronyms = ["ONU"]
model_url = "http://127.0.0.1"  # URL del servidor del modelo LLaMA
model_port = 8080  # Puerto del servidor del modelo LLaMA

module = AcronymGeneratorModule(model_url=model_url, model_port=model_port)
prediction = module.forward(acronyms)
print(prediction.TEXTO_GENERADO)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: /generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa3845e98a0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [63]:
# Para comprobar la conexión al servidor LLaMA
!curl -X POST http://127.0.0.1:8080/generate -d '{"prompt": "Hello"}'

curl: (7) Failed to connect to 127.0.0.1 port 8080 after 0 ms: Connection refused


In [ ]:
'''
filtered_samples = samples_train[samples_train['answers'].apply(lambda x: len(x) > 0)]
df_unique_titles = filtered_samples.drop_duplicates(subset='title', keep='first')
df_unique_titles.reset_index(drop=True, inplace=True)

new_samples = []
for idx, el in df_unique_titles.iterrows():
    if idx % 100 == 0:
        print(f"-- -- Processing index {idx} out of {len(df_unique_titles)}")
    cherry_picker = negator(question=el.question, golden_answer=el.answers[0]["text"])
    new_samples.append(
        [
            el.question, # question
            el.answers[0]["text"], # answer1
            cherry_picker.CHERRY_ANSWER, # answer2
            "CONTRADICTION", # label
            cherry_picker.RATIONALE, # rationale
        ]
    )

# Convert new_samples to a DataFrame if needed
new_samples_df = pd.DataFrame(new_samples, columns=['question', 'answer1', 'answer2', 'label', 'rationale'])
'''